In [1]:
import seaborn as sns
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.linear_model import LinearRegression #remember, this is linear regression model from sk-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

Bag-of-words counts words in a text cell and does this for each value provided. It provides insight into how often certain words reoccure. 

The Naïve Bayes model calculates the probability that a given text cell belongs to one of the provided categories.

Both models combined provide the oppertunity to classify text by their occurrance and calculate the probability to one of the given categories.

In [12]:
df = pd.read_csv('/Users/tim/Downloads/Assignment text mining - data clothing reviews.csv')
#df = df.dropna() #first get rid of rows with empty cells
df.head(5)

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


Only text reviews of dresses are needed, therefore the rest of the df is not required. I'm creating a subset for the sklearn library to predict the Review Text of only the dresses.

In [16]:
dfsub = df.loc[(df['Class Name'] == 'Dresses')] #code that takes all rows with the measure 'dresses'
#df_subset = df_subset[['Review Text', 'Rating', 'Class Name']] 
dfsub = df_subset[['Review Text', 'Rating']] #simplifying the subset for the Sklearn library

#change ratings to positive and negative
dfsub.loc[:, 'Rating'].replace([4, 5], ['Positive', 'Positive'], inplace=True)
dfsub.loc[:, 'Rating'].replace([1, 2, 3], ['Negative', 'Negative', 'Negative'], inplace=True)

dfsub.head(10)

,Review Text,Rating
1,Love this dress! it's sooo pretty. i happene...,Positive
2,I had such high hopes for this dress and reall...,Negative
5,"I love tracy reese dresses, but this one is no...",Negative
8,I love this dress. i usually get an xs but it ...,Positive
9,"I'm 5""5' and 125 lbs. i ordered the s petite t...",Positive
10,Dress runs small esp where the zipper area run...,Negative
11,This dress is perfection! so pretty and flatte...,Positive
12,More and more i find myself reliant on the rev...,Positive
14,This is a nice choice for holiday gatherings. ...,Negative
19,I love the look and feel of this tulle dress. ...,Positive


In [23]:
from sklearn.feature_extraction.text import CountVectorizer #The CountVectorizer object

text = dfsub['Review Text'].values.astype('U') #Taking the text from the df. We need to convert it to Unicode

vect = CountVectorizer(stop_words='english') #Create the CV object, with English stop words
vect = vect.fit(text) #We fit the model with the words from the review text

feature_names = vect.get_feature_names() #Get the words from the vocabulary
print(f"There are {len(feature_names)} words in the vocabulary. A selection: {feature_names[300:400]}") 

There are 8080 words in the vocabulary. A selection: ['6am', '6ft', '6in', '6inches', '6p', '6r', '6th', '70', '70s', '79', '80', '80f', '80s', '85', '89', '8in', '8p', '8petite', '90', '90s', '92', '95', '95lb', '95lbs', '96', '97', '98', '99', '9lower', '_______', '________', '_________', '__________', '___________', '____________', '_____________', '______________', '_________________', 'a34b', 'aa', 'aaaahs', 'aaahed', 'aame', 'abby', 'abdomen', 'abdominal', 'ability', 'abject', 'able', 'abo', 'abou', 'abroad', 'abruptly', 'absence', 'absolute', 'absolutely', 'absolutley', 'absoluty', 'abstract', 'absurdly', 'abt', 'abundance', 'abundant', 'abused', 'ac', 'accent', 'accented', 'accenting', 'accents', 'accentuate', 'accentuated', 'accentuates', 'accentuating', 'accept', 'acceptable', 'accepted', 'access', 'accessories', 'accessorize', 'accessorized', 'accessorizes', 'accessory', 'accident', 'accidentally', 'acco', 'accommodate', 'accommodated', 'accommodates', 'accommodating', 'acco

It's possible to see a range of the feature_names, above we see the transition from numbers to words starting with A.

In [29]:
docu_feat = vect.transform(text) #The transform method from the CountVectorizer object creates the matrix
print(docu_feat[0:50,0:50]) #print a small part of the matrix: the first 50 words & documents

  (2, 8)	1
  (20, 38)	1
  (21, 4)	1
  (21, 45)	1
  (22, 12)	1
  (26, 40)	1
  (35, 12)	2
  (39, 31)	1


In [32]:
#Code below is only for visual purpose, will take a lot of time and is redundant for the models.

#Make a regular matrix out of docu_feat, make it into a DataFrame and concatenate it along the columns
#rev_words = pd.concat([dfsub.reset_index(), pd.DataFrame(docu_feat.toarray())], axis=1)
#rev_words.head(5)

In [35]:
from sklearn.naive_bayes import MultinomialNB #import Naive bayes

x = docu_feat
y = dfsub['Rating']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1) #split the data, store it into different variables

clf = MultinomialNB()
clf.fit(x_train, y_train)

y_test_p = clf.predict(x_test)
print(clf.score(x_test, y_test))
print(clf.classes_)

0.8554852320675106
['Negative' 'Positive']


taking the oppertunity to also print the training data, just to see the gap between the training models.

In [43]:
y_train_p = clf.predict(x_train)
print(clf.score(x_train, y_train))
print(clf.classes_)

0.9274248247795613
['Negative' 'Positive']


Evaluation provides 86% percent accuracy of prediction, which is quite a lot. With the training data going as far as 92%. My conclusion is that the accuracy is quite high, as guessing will provide you already with a 50% accuracy.

In [48]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_test_p) #creates a "confusion matrix"
#make a dataframe.
conf_matrix = pd.DataFrame(cm, index=['Negative', 'Positive'], columns = ['Negative_p', 'Positive_p']) 
conf_matrix

,Negative_p,Positive_p
Negative,282,182
Positive,92,1340


In [49]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_test_p)) #creates a confusion report

              precision    recall  f1-score   support

    Negative       0.75      0.61      0.67       464
    Positive       0.88      0.94      0.91      1432

    accuracy                           0.86      1896
   macro avg       0.82      0.77      0.79      1896
weighted avg       0.85      0.86      0.85      1896



Plot the comments of the review text and add the rating and concluding the the probability.



In [55]:
for i in range(20): #get the predictions of text of 20 rows
    prob = clf.predict_proba(x[i]) 
    print(df_subset.iloc[i,0])
    print(i, f"The rating is: {df_subset.iloc[i,1]}")
    print(f"Negative: {prob[0,0]}, Positive: {prob[0,1]}")

Love this dress!  it's sooo pretty.  i happened to find it in a store, and i'm glad i did bc i never would have ordered it online bc it's petite.  i bought a petite and am 5'8".  i love the length on me- hits just a little below the knee.  would definitely be a true midi on someone who is truly petite.
0 The rating is: 5
Negative: 5.337412695289174e-05, Positive: 0.9999466258730467
I had such high hopes for this dress and really wanted it to work for me. i initially ordered the petite small (my usual size) but i found this to be outrageously small. so small in fact that i could not zip it up! i reordered it in petite medium, which was just ok. overall, the top half was comfortable and fit nicely, but the bottom half had a very tight under layer and several somewhat cheap (net) over layers. imo, a major design flaw was the net over layer sewn directly into the zipper - it c
1 The rating is: 3
Negative: 0.9999990978503296, Positive: 9.021496679830375e-07
I love tracy reese dresses, but t

On review 13 we short and mixed review, overall I would classify this review as negative and something for the designers to take into account when designing the new collection. The model has predicted it positive, this is most likely due to the words: Love & easily. In addition, it's quite a short comment, so the models doesn't have too much input.